In [50]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
from os.path import isfile

In [337]:
def get_url_resp(country="Worldwide"):
    base_url = "https://www.worldometers.info/coronavirus/"
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0"}
    if country =="Worldwide":
        url = base_url
    else:
        url =base_url+"country/"+country+"/"
    response = requests.get(url, headers = headers)
    html = response.text
    return BeautifulSoup(html,"html.parser")

def format_case_numbers(soup,country="Worldwide"):
    if country == "Worldwide":
        soup_selector = soup.select(".col-md-6")
        val = [soup_selector[2].find_all(type="text/javascript")[0],soup_selector[3].find_all(type="text/javascript")[0]]
    else:
        soup_selector = soup.find_all("script")
        new_soup =[]
        for i in range(len(soup_selector)):
            if 'categories' in str(soup_selector[i]):
                new_soup.append(soup_selector[i])
        val = new_soup[:2]
    return val

def get_case_numbers_from_soup(val):#,country="Worldwide",status="Cases"):
    dates = val.get_text().rsplit("categories",1)[1].split("[")[1].split("]")[0]
    case_num = val.get_text().rsplit("data",1)[1].split("[")[1].split("]")[0]
    case_num_formatted=case_num.split(',')
    dates_formatted=dates.split(',')
    case_num_lst=[]
    date_lst = []
    # for lst in case_num_lst,date_lst:
    #     lst.append(country)
    #     lst.append(status)

    for i in range(len(case_num_formatted)):
        if case_num_formatted[i] == "null":
            case_num_input = 0
        else:
            case_num_input = int(case_num_formatted[i])
        case_num_lst.append(case_num_input)

    for i in dates_formatted:
        date_lst.append(i.replace('"',''))
    return zip(date_lst,case_num_lst)

def get_case_dictionary(country="Worldwide"):#,status="Cases"):
    soup = get_url_resp(country)
    val = format_case_numbers(soup,country)
    statuses_tuple = ("Cases","Deaths")
    statuses = {
        "Cases":0,
        "Deaths":1,
    }
    lst=[]
    for sta in statuses_tuple:
        lst.append(dict(get_case_numbers_from_soup(val[statuses[sta]])))
    return lst
    #return dict(get_case_numbers_from_soup(val[statuses[status]]))#,country,status)
    # return get_case_numbers_from_soup(val[statuses[status]],country,status)


In [339]:
Countries = ("Worldwide","US","Italy","China","Spain","Germany","France","UK","India","South-Korea")
statuses_tuple = ("Cases","Deaths")
lst=[[],[]]
for country in Countries:
    for sta in statuses_tuple:
        lst[0].append(country)
        lst[1].append(sta)
### WORKS TO HERE ###
# lst = [["Worldwide","US"],["Cases","Cases"]]

data=[]
for country in Countries:
    country_data=get_case_dictionary(country=country)
    print(country_data)
    data.extend(country_data)
print(data)

[['Worldwide', 'Worldwide', 'US', 'US', 'Italy', 'Italy', 'China', 'China', 'Spain', 'Spain', 'Germany', 'Germany', 'France', 'France', 'UK', 'UK', 'India', 'India', 'South-Korea', 'South-Korea'], ['Cases', 'Deaths', 'Cases', 'Deaths', 'Cases', 'Deaths', 'Cases', 'Deaths', 'Cases', 'Deaths', 'Cases', 'Deaths', 'Cases', 'Deaths', 'Cases', 'Deaths', 'Cases', 'Deaths', 'Cases', 'Deaths']]
[{'Jan 22': 580, 'Jan 23': 845, 'Jan 24': 1317, 'Jan 25': 2015, 'Jan 26': 2800, 'Jan 27': 4581, 'Jan 28': 6058, 'Jan 29': 7813, 'Jan 30': 9823, 'Jan 31': 11950, 'Feb 01': 14553, 'Feb 02': 17391, 'Feb 03': 20630, 'Feb 04': 24545, 'Feb 05': 28266, 'Feb 06': 31439, 'Feb 07': 34876, 'Feb 08': 37552, 'Feb 09': 40553, 'Feb 10': 43099, 'Feb 11': 45134, 'Feb 12': 59287, 'Feb 13': 64438, 'Feb 14': 67100, 'Feb 15': 69197, 'Feb 16': 71329, 'Feb 17': 73332, 'Feb 18': 75184, 'Feb 19': 75700, 'Feb 20': 76677, 'Feb 21': 77673, 'Feb 22': 78651, 'Feb 23': 79205, 'Feb 24': 80087, 'Feb 25': 80828, 'Feb 26': 81820, 'Feb 27'

ValueError: invalid literal for int() with base 10: '"nan"'

In [325]:
tuples = list(zip(*lst))
index = pd.MultiIndex.from_tuples(tuples)
cols = list(data[0].keys())
data_types = {val:pd.Int64Dtype() for val in cols}

In [327]:
df=pd.DataFrame(data, index=index, columns=cols)#,dtype=data_types)
df

,,Jan 22,Jan 23,Jan 24,Jan 25,Jan 26,Jan 27,Jan 28,Jan 29,Jan 30,Jan 31,...,Mar 19,Mar 20,Mar 21,Mar 22,Mar 23,Mar 24,Mar 25,Mar 26,Mar 27,Mar 28
Worldwide,Cases,580.0,845.0,1317.0,2015.0,2800.0,4581.0,6058.0,7813.0,9823.0,11950.0,...,244902,275550,304979,337459,378830,422574,471035,531865,596366,663127
US,Cases,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13779,19367,24192,33592,43781,54856,68211,85435,104126,123578
